In [1]:
import sys
import pandas as pd
import numpy as np

# Add the project root directory to Python path
project_root = r'c:\Users\pedro.hun\Documents\repos\option_monitor2'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.orchestrators.df_creator import DFCreator
from src.orchestrators.single_expiry_fitting import SingleExpiry

In [2]:
df = DFCreator()
final_df = df.apply_greeks()

Could not adjust IV limits to bracket root
Market Price: 11.405, Price at Low Vol: 0.0, Price at High Vol: 9.32198147474064 for strike 18.47
Invalid price range for IV calculation
Could not adjust IV limits to bracket root
Market Price: 11.16, Price at Low Vol: 0.0, Price at High Vol: 9.424757324880005 for strike 18.72
Invalid price range for IV calculation
Could not adjust IV limits to bracket root
Market Price: 10.91, Price at Low Vol: 0.0, Price at High Vol: 9.526555792723485 for strike 18.97
Invalid price range for IV calculation
Could not adjust IV limits to bracket root
Market Price: 10.665, Price at Low Vol: 0.0, Price at High Vol: 9.627388783394933 for strike 19.22
Invalid price range for IV calculation
Could not adjust IV limits to bracket root
Market Price: 10.42, Price at Low Vol: 0.0, Price at High Vol: 9.727268117326899 for strike 19.47
Invalid price range for IV calculation
Could not adjust IV limits to bracket root
Market Price: 10.175, Price at Low Vol: 0.0, Price at Hi

In [3]:
final_df

,bid,ask,lastPrice,Strike,openInterest,volume,ticker,Expiry,IV,SpotPrice,...,CalcIV,W,D1,D2,BSPrice,Vega,Theta,Rho,Delta,Gamma
88,11.37,11.44,11.26,18.47,16100.0,500.0,PETRK190,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,11.13,11.19,11.78,18.72,8200.0,3000.0,PETRK193,21/11/2025,1.391983,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,10.88,10.94,10.93,18.97,188700.0,1300.0,PETRK30,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,10.64,10.69,10.56,19.22,15100.0,1400.0,PETRK198,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,10.39,10.45,10.30,19.47,137600.0,1200.0,PETRK200,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.96,0.97,0.96,30.22,5937400.0,969300.0,PETRW308,21/11/2025,0.250425,29.68,...,0.300813,0.007900,0.110024,0.021143,0.965,3.477414,-3.940021,-1.114042,-0.456195,0.150318
414,1.09,1.11,1.10,30.47,2473800.0,377100.0,PETRW309,21/11/2025,0.250549,29.68,...,0.373464,0.012176,0.182660,0.072313,1.100,3.440646,-5.377115,-1.076667,-0.427532,0.119796
415,1.24,1.26,1.24,30.72,5431500.0,587000.0,PETRW312,21/11/2025,0.247536,29.68,...,0.449655,0.017652,0.233818,0.100959,1.250,3.404187,-6.816633,-1.059272,-0.407563,0.098443
416,1.40,1.43,1.42,30.97,4988600.0,117700.0,PETRW314,21/11/2025,0.253410,29.68,...,0.529948,0.024518,0.272081,0.115498,1.415,3.371398,-8.291336,-1.054499,-0.392780,0.082723


In [4]:
final_df

,bid,ask,lastPrice,Strike,openInterest,volume,ticker,Expiry,IV,SpotPrice,...,CalcIV,W,D1,D2,BSPrice,Vega,Theta,Rho,Delta,Gamma
88,11.37,11.44,11.26,18.47,16100.0,500.0,PETRK190,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,11.13,11.19,11.78,18.72,8200.0,3000.0,PETRK193,21/11/2025,1.391983,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,10.88,10.94,10.93,18.97,188700.0,1300.0,PETRK30,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,10.64,10.69,10.56,19.22,15100.0,1400.0,PETRK198,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,10.39,10.45,10.30,19.47,137600.0,1200.0,PETRK200,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.96,0.97,0.96,30.22,5937400.0,969300.0,PETRW308,21/11/2025,0.250425,29.68,...,0.300813,0.007900,0.110024,0.021143,0.965,3.477414,-3.940021,-1.114042,-0.456195,0.150318
414,1.09,1.11,1.10,30.47,2473800.0,377100.0,PETRW309,21/11/2025,0.250549,29.68,...,0.373464,0.012176,0.182660,0.072313,1.100,3.440646,-5.377115,-1.076667,-0.427532,0.119796
415,1.24,1.26,1.24,30.72,5431500.0,587000.0,PETRW312,21/11/2025,0.247536,29.68,...,0.449655,0.017652,0.233818,0.100959,1.250,3.404187,-6.816633,-1.059272,-0.407563,0.098443
416,1.40,1.43,1.42,30.97,4988600.0,117700.0,PETRW314,21/11/2025,0.253410,29.68,...,0.529948,0.024518,0.272081,0.115498,1.415,3.371398,-8.291336,-1.054499,-0.392780,0.082723


In [5]:
final_df[final_df["volume"]>100]

,bid,ask,lastPrice,Strike,openInterest,volume,ticker,Expiry,IV,SpotPrice,...,CalcIV,W,D1,D2,BSPrice,Vega,Theta,Rho,Delta,Gamma
88,11.37,11.44,11.26,18.47,16100.0,500.0,PETRK190,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,11.13,11.19,11.78,18.72,8200.0,3000.0,PETRK193,21/11/2025,1.391983,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,10.88,10.94,10.93,18.97,188700.0,1300.0,PETRK30,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,10.64,10.69,10.56,19.22,15100.0,1400.0,PETRK198,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
92,10.39,10.45,10.30,19.47,137600.0,1200.0,PETRK200,21/11/2025,0.078125,29.68,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,0.96,0.97,0.96,30.22,5937400.0,969300.0,PETRW308,21/11/2025,0.250425,29.68,...,0.300813,0.007900,0.110024,0.021143,0.965,3.477414,-3.940021,-1.114042,-0.456195,0.150318
414,1.09,1.11,1.10,30.47,2473800.0,377100.0,PETRW309,21/11/2025,0.250549,29.68,...,0.373464,0.012176,0.182660,0.072313,1.100,3.440646,-5.377115,-1.076667,-0.427532,0.119796
415,1.24,1.26,1.24,30.72,5431500.0,587000.0,PETRW312,21/11/2025,0.247536,29.68,...,0.449655,0.017652,0.233818,0.100959,1.250,3.404187,-6.816633,-1.059272,-0.407563,0.098443
416,1.40,1.43,1.42,30.97,4988600.0,117700.0,PETRW314,21/11/2025,0.253410,29.68,...,0.529948,0.024518,0.272081,0.115498,1.415,3.371398,-8.291336,-1.054499,-0.392780,0.082723


In [6]:
# final_df = final_df[final_df["OptionType"] == "call"]

In [7]:
for x in final_df['TTE_days'].unique():
    fitting = SingleExpiry(days_to_expiry=x, df=final_df)
    fitting.plot_svi()

Found 123 options with 22 days to expiry
Moneyness range: -0.214 to 0.487
Initial guess for SVI parameters: [np.float64(nan), np.float64(nan), np.float64(nan), np.float64(0.4865270635017356), np.float64(0.17517461425073902)]
SVI fitting failed: Input contains NaN.
Plotting failed: Input contains NaN.


c:\Users\pedro.hun\Documents\repos\option_monitor2\src\core\interpolation_single_expiry\svi_redo.py:153: OptimizeWarning: Unknown solver options: ftol, gtol
  p0 = sop.minimize(obj_fun, x0=initial_guess, method=method, bounds=bounds, options={'maxiter':10000, 'ftol':1e-8, 'gtol':1e-7})


ValueError: Input contains NaN.